# Lab 12-2 many to one stacked
### sentence sentiment classification 
* many to one
* variable input sequence length
* stacked
* drop out

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pprint import pprint
tf.set_random_seed(777)

### Prepairing dataset

In [2]:
sentences = ['What I cannot create, I do not understand.',
             'Intellecuals solve problems, geniuses prevent them',
             'A person who never made a mistake never tied anything new.',
             'The same equations have the same solutions.']
y_data = [1,0,0,1] # 1은 리처드파인만 한 말, 0은 아인슈타인이 한 말

In [3]:
char_set = ['<pad>'] + sorted(list(set(''.join(sentences))))
idx2char = {idx : char for idx, char in enumerate(char_set)}
char2idx = {char : idx for idx, char in enumerate(char_set)}

print(idx2char)
print(char2idx)

{0: '<pad>', 1: ' ', 2: ',', 3: '.', 4: 'A', 5: 'I', 6: 'T', 7: 'W', 8: 'a', 9: 'b', 10: 'c', 11: 'd', 12: 'e', 13: 'g', 14: 'h', 15: 'i', 16: 'k', 17: 'l', 18: 'm', 19: 'n', 20: 'o', 21: 'p', 22: 'q', 23: 'r', 24: 's', 25: 't', 26: 'u', 27: 'v', 28: 'w', 29: 'y'}
{'<pad>': 0, ' ': 1, ',': 2, '.': 3, 'A': 4, 'I': 5, 'T': 6, 'W': 7, 'a': 8, 'b': 9, 'c': 10, 'd': 11, 'e': 12, 'g': 13, 'h': 14, 'i': 15, 'k': 16, 'l': 17, 'm': 18, 'n': 19, 'o': 20, 'p': 21, 'q': 22, 'r': 23, 's': 24, 't': 25, 'u': 26, 'v': 27, 'w': 28, 'y': 29}


In [4]:
x_data = list(map(lambda sentence : [char2idx.get(char) for char in sentence], sentences))
x_data_len = list(map(lambda sentence : len(sentence), sentences))

print(x_data)
print(x_data_len)

[[7, 14, 8, 25, 1, 5, 1, 10, 8, 19, 19, 20, 25, 1, 10, 23, 12, 8, 25, 12, 2, 1, 5, 1, 11, 20, 1, 19, 20, 25, 1, 26, 19, 11, 12, 23, 24, 25, 8, 19, 11, 3], [5, 19, 25, 12, 17, 17, 12, 10, 26, 8, 17, 24, 1, 24, 20, 17, 27, 12, 1, 21, 23, 20, 9, 17, 12, 18, 24, 2, 1, 13, 12, 19, 15, 26, 24, 12, 24, 1, 21, 23, 12, 27, 12, 19, 25, 1, 25, 14, 12, 18], [4, 1, 21, 12, 23, 24, 20, 19, 1, 28, 14, 20, 1, 19, 12, 27, 12, 23, 1, 18, 8, 11, 12, 1, 8, 1, 18, 15, 24, 25, 8, 16, 12, 1, 19, 12, 27, 12, 23, 1, 25, 15, 12, 11, 1, 8, 19, 29, 25, 14, 15, 19, 13, 1, 19, 12, 28, 3], [6, 14, 12, 1, 24, 8, 18, 12, 1, 12, 22, 26, 8, 25, 15, 20, 19, 24, 1, 14, 8, 27, 12, 1, 25, 14, 12, 1, 24, 8, 18, 12, 1, 24, 20, 17, 26, 25, 15, 20, 19, 24, 3]]
[42, 50, 58, 43]


In [5]:
# padding
max_sequence = 55
x_data = pad_sequences(sequences = x_data, maxlen = max_sequence,
                       padding = 'post', truncating = 'post')

# 데이터 형태 확인
print(x_data)
print(x_data_len)
print(y_data)

[[ 7 14  8 25  1  5  1 10  8 19 19 20 25  1 10 23 12  8 25 12  2  1  5  1
  11 20  1 19 20 25  1 26 19 11 12 23 24 25  8 19 11  3  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 5 19 25 12 17 17 12 10 26  8 17 24  1 24 20 17 27 12  1 21 23 20  9 17
  12 18 24  2  1 13 12 19 15 26 24 12 24  1 21 23 12 27 12 19 25  1 25 14
  12 18  0  0  0  0  0]
 [ 4  1 21 12 23 24 20 19  1 28 14 20  1 19 12 27 12 23  1 18  8 11 12  1
   8  1 18 15 24 25  8 16 12  1 19 12 27 12 23  1 25 15 12 11  1  8 19 29
  25 14 15 19 13  1 19]
 [ 6 14 12  1 24  8 18 12  1 12 22 26  8 25 15 20 19 24  1 14  8 27 12  1
  25 14 12  1 24  8 18 12  1 24 20 17 26 25 15 20 19 24  3  0  0  0  0  0
   0  0  0  0  0  0  0]]
[42, 50, 58, 43]
[1, 0, 0, 1]


### Creating graph

In [6]:
# input
data = tf.data.Dataset.from_tensor_slices((x_data_len, x_data, y_data))
data = data.shuffle(buffer_size = 4)
data = data.batch(batch_size = 2)
data_iter = data.make_initializable_iterator()
x_mb_len, x_mb, y_mb = data_iter.get_next()
keep_prob = tf.placeholder(dtype = tf.float32)

In [7]:
# hyper-parameters for lstm (many to one), one-hot encoding
n_of_classes = 2
hidden_dims = [32, 32]

one_hot_encoding = tf.eye(num_rows = len(idx2char), dtype = tf.float32)
one_hot_encoding = tf.get_variable(name = 'one_hot_encoding',
                                   initializer = one_hot_encoding, trainable = False)
x_mb_batch = tf.nn.embedding_lookup(params = one_hot_encoding, ids = x_mb)

lstm_cells = []

for hidden_dim in hidden_dims:
    lstm_cell = tf.nn.rnn_cell.LSTMCell(num_units = hidden_dim, activation = tf.nn.tanh)
    lstm_cell = tf.nn.rnn_cell.DropoutWrapper(cell = lstm_cell, output_keep_prob = keep_prob)
    lstm_cells.append(lstm_cell)
else:
    lstm_cells = tf.nn.rnn_cell.MultiRNNCell(cells = lstm_cells)
                                
_, states = tf.nn.dynamic_rnn(cell = lstm_cells, inputs = x_mb_batch, sequence_length = x_mb_len,
                              dtype = tf.float32)

In [8]:
score = keras.layers.Dense(units = n_of_classes)(states[1].h)
loss = tf.losses.sparse_softmax_cross_entropy(labels = y_mb, logits = score)
prediction = tf.argmax(input = score, axis = -1)

In [9]:
# training
lr = .1
opt = tf.train.AdamOptimizer(learning_rate = lr)
training_op = opt.minimize(loss = loss)

### Training

In [10]:
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config = sess_config)
sess.run(tf.global_variables_initializer())

In [11]:
epochs = 20
tr_loss_hist = []

for epoch in range(epochs):
    
    sess.run(data_iter.initializer)
    avg_tr_loss = 0
    step = 0
    
    try:
        while True:
            _, tr_loss = sess.run([training_op, loss], feed_dict = {keep_prob : .7})
            avg_tr_loss += tr_loss
            step += 1
    except:
        avg_tr_loss /= step
        tr_loss_hist.append(avg_tr_loss)
        
    print('epoch : {:3}, loss : {:.3f}'.format(epoch + 1, avg_tr_loss))

epoch :   1, loss : 1.463
epoch :   2, loss : 1.060
epoch :   3, loss : 0.716
epoch :   4, loss : 0.584
epoch :   5, loss : 0.726
epoch :   6, loss : 0.767
epoch :   7, loss : 0.411
epoch :   8, loss : 0.539
epoch :   9, loss : 0.754
epoch :  10, loss : 0.917
epoch :  11, loss : 0.710
epoch :  12, loss : 0.689
epoch :  13, loss : 0.664
epoch :  14, loss : 0.487
epoch :  15, loss : 0.279
epoch :  16, loss : 0.326
epoch :  17, loss : 0.199
epoch :  18, loss : 0.050
epoch :  19, loss : 0.132
epoch :  20, loss : 0.294


### Accuracy

In [12]:
yhat = sess.run(prediction, feed_dict = {x_mb : x_data, x_mb_len : x_data_len, keep_prob : 1.})

In [13]:
print('accuracy : {:.2%}'.format(np.mean(yhat == y_data)))

accuracy : 100.00%
